# Face Recognition using LBP

In [35]:
from skimage import feature
import cv2
import numpy as np
import os

# To calculate a normalized histogram 
from scipy.stats import itemfreq
from sklearn.preprocessing import normalize

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

In [57]:
# reading in the images
sourceFolder = "face_datasets/aligned_AVR2_data/aligned_data/"
folders = os.listdir(sourceFolder)

allImages = []
allLabels = []

# for every folder
for folder in folders:
    # folder path to that folder
    folder_path = sourceFolder + folder
    
    # read in the image names for that folder
    image_names = os.listdir(folder_path)
    
    # read in the individual images
    for img in image_names:
        if (img == "Thumbs.db"):
            continue
        allImages.append(cv2.imread(folder_path + "/" + img, cv2.IMREAD_GRAYSCALE))
        allLabels.append(folder)
    
    # flatenning and vertically stacking all images
    vStack = np.asarray(allImages[0]).reshape(1, -1)
    for img in allImages[1:]:
        vStack = np.vstack((vStack, np.asarray(img).reshape(1, -1)))

In [58]:
# finding the feature histogram
data = []
for img in allImages:
    numPoints=90
    radius=15
    lbp = feature.local_binary_pattern(img, P=numPoints, R=radius, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, numPoints + 3), range=(0, numPoints + 2))
    eps=1e-7
    hist = hist.astype("float")
    hist /= (hist.sum() + eps)
    data.append(hist)

In [59]:
# splitting into training and testing datasets
trainImages, testImages, trainLabels, testLabels = train_test_split(data, 
            allLabels, stratify=allLabels, test_size=0.2, random_state=42)

In [60]:
# training the SVM classifier
model = LinearSVC(C=116, random_state=42)
model.fit(trainImages, trainLabels)

/home/vasu/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=116, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
     verbose=0)

In [61]:
# making a prediction on test data
predicted = model.predict(testImages)

In [62]:
# finding the accuracy
accuracy_score(testLabels, predicted) * 100

29.927007299270077

# Making Top 3 and Top 10 predictions

In [63]:
confidence = model.decision_function(testImages)

classes = model.classes_

In [64]:
# for every test image we order the confidence scores in descreasing order with their labels
a=[]
for i in range(len(testImages)):
    t = zip(confidence[i], classes.copy())
    t = sorted(t, reverse=True)
    x, y = zip(*t)
    a.append(y)
    
# Top 3
accuracy_count = 0
for i in range(len(testImages)):
    if (testLabels[i] in a[i][:3]):
        accuracy_count += 1
        
print ("Top 3 Accuracy: " + str((accuracy_count / len(testImages)) * 100))

# Top 10
accuracy_count = 0
for i in range(len(testImages)):
    if (testLabels[i] in a[i][:10]):
        accuracy_count += 1
        
print ("Top 10 Accuracy: " + str((accuracy_count / len(testImages)) * 100))

Top 3 Accuracy: 54.01459854014598
Top 10 Accuracy: 76.64233576642336
